# TFG

Primero importamos las librerías

In [2]:
import pandas as pd
import numpy as np

Importamos los datos, e imprimimos los primeros para comprobar que se han importado correctamente.

In [5]:
df = pd.read_csv('SolUsd.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-10,0.832005,1.313487,0.694187,0.951054,0.951054,87364276
1,2020-04-11,0.951054,1.049073,0.765020,0.776819,0.776819,43862444
2,2020-04-12,0.785448,0.956670,0.762426,0.882507,0.882507,38736897
3,2020-04-13,0.890760,0.891603,0.773976,0.777832,0.777832,18211285
4,2020-04-14,0.777832,0.796472,0.628169,0.661925,0.661925,16747614
